## Create FHIR R4 SearchParameter Resource

Create FHIR R4 SearchParameter Resource, Quick start text, and Searchparameter list using the python fhir client

Source data is in excel file

### Prerequisites:

- Python 3.7 or greater


### Import FHIRR4Client and other libraries

In [1]:
%config IPCompleter.greedy=True

In [2]:
from fhirclient.r4models.fhirabstractbase import FHIRValidationError
from fhirclient.r4models import searchparameter as SP
from fhirclient.r4models import capabilitystatement as CS
from fhirclient.r4models import bundle as B
from fhirclient.r4models import narrative as N
import fhirclient.r4models.identifier as I
import fhirclient.r4models.coding as C
import fhirclient.r4models.codeableconcept as CC
import fhirclient.r4models.fhirdate as D
import fhirclient.r4models.extension as X
import fhirclient.r4models.contactdetail as CD
from json import dumps, loads, load
from requests import get, post, put
import os
from pathlib import Path
from csv import reader as csvreader
from IPython.display import display as Display, HTML, Markdown
from pprint import pprint
from collections import namedtuple
from pandas import *
from datetime import datetime
from jinja2 import Environment, FileSystemLoader, select_autoescape
import R4sp_summary_list as sp_map
from stringcase import snakecase, titlecase, pascalcase
from itertools import zip_longest
from openpyxl import load_workbook
import R4sp_summary_list as sp_map
from itertools import zip_longest
from openpyxl import load_workbook
from lxml import etree
from commonmark import commonmark

####  Assign Global Variables


Here is where we assign all the global variables for this example such as the local paths for file input and output

##### Need to update:
- base_id
- paths
- canonical

In [6]:
#******************** Need to update for each IG *************************************************
fhir_base_url = 'http://hl7.org/fhir/R4/'
base_id = "US-Core"
canon_base = "http://hl7.org/fhir/us/core/"
ig_folder = 'US-Core-R4'
publisher = 'HL7 International - US Realm Steering Committee'
publisher_endpoint = dict(
                        system = 'url',
                        value = 'http://www.hl7.org/Special/committees/usrealm/index.cfm'
                        ) 

ig_source_path = "//ERICS-AIR-2/ehaas/Documents/FHIR/US-Core-R4/input/"  # for pc
#ig_source_path = "/Users/ehaas/Documents/FHIR/US-Core-R4/input/"    # for mac

write_path = '' # temp path


#spdef_json = 'C:/Users/Eric/Documents/HL7/FHIR/BUILD_EDIT_FILES/R4_Definitions/search-parameters.json'
#spdef_json ='/Users/ehaas/Downloads/definitionsR4.json/search-parameters.json' # v  4.0.1
spdef_json = 'sp_defs_4.0.1.json'  #v 4.0.1 does not have mods!! use this for v4

skip_types = ['Questionnaire',]

### limit to these profiles ( DOES NOT WORK If >1 Profile of same TYPE - Comment out to do  Quick starts for all profiles ####
whitelist = ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-allergyintolerance',] #['http://hl7.org/fhir/us/core/StructureDefinition/us-core-organization',] #'http://hl7.org/fhir/us/core/StructureDefinition/us-core-diagnosticreport-note']

validate_flag = False
#***********************************************************************************

def markdown(text, *args, **kwargs):
    return commonmark(text, *args, **kwargs)

env = Environment(
    loader=FileSystemLoader(searchpath = ''),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'])
    )


env.filters['markdown'] = markdown



md_template = ['quick_start.j2', 'sp_list_page.j2', 'cs_search_documentation.j2','sp_narrative.j2']


fhir_term_server = 'http://test.fhir.org/r4'

#profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient


none_list = ['', ' ', 'none', 'n/a', 'N/A', 'N', 'False', 'FALSE']
sep_list = (',', ';', ' ', ', ', '; ')

### Write to File

In [7]:
 def write_file(path,name,data): # write file
    with open(f'{write_path}{path}{name}', 'w', newline='\n') as f:
        f.write(data)

### validate

In [8]:
# *********************** validate Resource as Dict ********************************

def validate(r):

    fhir_test_server = 'http://test.fhir.org/r4'

    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }
    
    params = {
    }
    
    r = post(f'{fhir_test_server}/{r["resourceType"]}/$validate', params = params, headers = headers, data = dumps(r))
    # return r.status_code
    # view  output
    # return (r.json()["text"]["div"])
    return r

### Get Search Parameter input data

In [9]:
in_path = f"{ig_source_path}resources_spreadsheets/"
#in_path =''

in_file ="uscore-server"

xls = ExcelFile(f'{in_path}{in_file}.xlsx')
df = read_excel(xls,'sps',na_filter = False)
df_combos = read_excel(xls,'sp_combos',na_filter = False)

df_combos

,index,base,profile,combo,combo_conf,types,fixed_kv,description,example,imp_note
0,1,!Encounter,http://hl7.org/fhir/us/core/StructureDefinitio...,"class,date",SHOULD,"date,token",,,,Fetches a bundle of all !Encounter resources m...
1,2,!Encounter,http://hl7.org/fhir/us/core/StructureDefinitio...,"class,date,patient",SHOULD,"date,reference,token",,,,Fetches a bundle of all !Encounter resources m...
2,3,!Encounter,http://hl7.org/fhir/us/core/StructureDefinitio...,"class,date,patient,type",SHOULD,"date,reference,token",,,,Fetches a bundle of all !Encounter resources m...
3,4,!Encounter,http://hl7.org/fhir/us/core/StructureDefinitio...,"class,date,type",SHOULD,"date,token",,,,Fetches a bundle of all !Encounter resources m...
4,5,Encounter,http://hl7.org/fhir/us/core/StructureDefinitio...,"class,patient",SHOULD,"reference,token",,support searching for all encounter for a pati...,GET [base]/Encounter?patient=example1&class= h...,Fetches a bundle of all Encounter resources ma...
5,6,!Encounter,http://hl7.org/fhir/us/core/StructureDefinitio...,"class,patient,status",SHOULD,"reference,token",,,,Fetches a bundle of all !Encounter resources m...
6,7,!Encounter,http://hl7.org/fhir/us/core/StructureDefinitio...,"class,patient,status,type",SHOULD,"reference,token",,,,Fetches a bundle of all !Encounter resources m...
7,8,!Encounter,http://hl7.org/fhir/us/core/StructureDefinitio...,"class,patient,type",SHOULD,"reference,token",,,,Fetches a bundle of all !Encounter resources m...
8,9,!Encounter,http://hl7.org/fhir/us/core/StructureDefinitio...,"class,status",SHOULD,token,,,,Fetches a bundle of all !Encounter resources m...
9,10,!Encounter,http://hl7.org/fhir/us/core/StructureDefinitio...,"class,status,type",SHOULD,token,,,,Fetches a bundle of all !Encounter resources m...


#### Defin SPs and Combos



In [10]:
try:
    data = [i for i in df.itertuples(index=True) 
                  if '!' not in i.base and i.profile in whitelist]
    combo_data = [i for i in df_combos.itertuples(index=True)
                  if '!' not in i.base and i.profile in whitelist]
except NameError:
    data = [i for i in df.itertuples(index=True) if '!' not in i.base]
    combo_data = [i for i in df_combos.itertuples(index=True) if '!' not in i.base]
    
r_type =  {d.base for d in data }

search_profiles = {i.profile:i.base for i in combo_data}

for d in data:
    print(f'Resource = {d.base}, Search Parameter = {d.code}, Exists = {d.exists}')
for c in combo_data:
    print(f'Resource = {c.base}, Combo Search Parameter = {c.combo}')

Resource = AllergyIntolerance, Search Parameter = clinical-status, Exists = Y
Resource = AllergyIntolerance, Search Parameter = patient, Exists = Y
Resource = AllergyIntolerance, Combo Search Parameter = patient,clinical-status


### update core SP with additional capabiliities


- Get definitions bundle and convert to python object for ease of notation
- use sp_map to map to Type + parameter
- If need to update SP Extract the SP based on the excel file

### load SP Mapping dictionary

output shows a single SP entry for a sanity check

In [21]:
p = Path(spdef_json)
b = B.Bundle(loads(p.read_text()), strict = False)
#sp = (i.resource for i in b.entry if i.resource.url == 'http://hl7.org/fhir/SearchParameter/Resource-id')
for i in b.entry:
    if 'clinical-patient' in i.resource.id:
          print(dumps(i.resource.as_json(), indent=4))

{
    "id": "clinical-patient",
    "extension": [
        {
            "url": "http://hl7.org/fhir/StructureDefinition/structuredefinition-standards-status",
            "valueCode": "trial-use"
        }
    ],
    "base": [
        "AllergyIntolerance",
        "CarePlan",
        "CareTeam",
        "ClinicalImpression",
        "Composition",
        "Condition",
        "Consent",
        "DetectedIssue",
        "DeviceRequest",
        "DeviceUseStatement",
        "DiagnosticReport",
        "DocumentManifest",
        "DocumentReference",
        "Encounter",
        "EpisodeOfCare",
        "FamilyMemberHistory",
        "Flag",
        "Goal",
        "ImagingStudy",
        "Immunization",
        "List",
        "MedicationAdministration",
        "MedicationDispense",
        "MedicationRequest",
        "MedicationStatement",
        "NutritionOrder",
        "Observation",
        "Procedure",
        "RiskAssessment",
        "ServiceRequest",
        "SupplyDelivery

## create updated SPs
- sp optional 'modifier' elements are listed as comma separated list of shalls and shoulds for each:
- multipleOr
- multipleOr_conf
- multipleAnd
- multipleAnd_conf
- shall_modifier
- should_modifier
- shall_comparator
- should_comparator
- shall_chain

 if not spedified then conformance is MAY 

- note that it starts out using the FHIRClient models but the switches to a dict structure to add the FHIR primitive type extensions.*

- create a nice XHTML narrative without spaces
   - stick the div in the body
   - the declaration must be  `<?xml version="1.0"?>`  and not `<?xml version="1.0" encoding="UTF-8"?>`

In [ ]:
def sp_expectation(conf=None):
    if not conf:
        conf = "MAY"
 
    x = X.Extension(dict(
    url = f'http://hl7.org/fhir/StructureDefinition/capabilitystatement-expectation',
    valueCode = conf
    ))
    x_dict = dict(
    extension = [x.as_json()]
    ) 
    return x_dict

def find_base_sp(d):
    if '!' not in d.base and d.update =='Y':
        #print(d.base, d.code)
        if d.code == '_id':
            sp = SP.SearchParameter(sp_for_id.as_json())
        else:
            sp = next(i.resource for i in b.entry if i.resource.code == d.code and d.base in i.resource.base)       
            sp = SP.SearchParameter(sp.as_json())
            #print(sp.description)
        return sp
    
def kebab_to_pascal(word):
    return ''.join(x.capitalize() for x in word.split('-'))

expect_note = '''
**NOTE**: This US Core SearchParameter definition extends the usage context of the
[Conformance expectation extension](http://hl7.org/fhir/R4/extension-capabilitystatement-expectation.html)
 - multipleAnd
 - multipleOr
 - comparator
 - modifier
 - chain'''

sp_list=[]
for d in data:
        print(f'========={d.base},{d.code}==============')
        sp = find_base_sp(d)
        #print(type(sp))
        if sp:
            #print(sp.url)
            # change id and url, publisher, and contact, draft etc
            sp.id = f'{base_id.lower()}-{d.base.lower()}-{d.code.replace("_","")}'  
            sp.extension = []
            sp.derivedFrom =sp.url
            sp.url = f'{canon_base}SearchParameter/{sp.id}'
            sp.publisher = publisher
            sp.contact = [CD.ContactDetail( {"telecom" : [ publisher_endpoint ] })]
            sp.date = D.FHIRDate(f'{datetime.utcnow().isoformat()}Z')
            sp.name = kebab_to_pascal(sp.id)
            sp.name = sp.name.replace('UsCore','USCore')
            sp.status = 'active'
            sp.base = [d.base]
            my_expression = [i.strip() for i in sp.expression.split('|') if i.strip().startswith(f'{d.base}.')]   
            sp.expression = '|'.join(my_expression)
            my_xpath = [i.strip() for i in sp.xpath.split('|') if i.strip().startswith(f'f:{d.base}/')]
            sp.xpath = '|'.join(my_xpath)  
            #print("---", sp.description)
            my_description = [i.strip() for i in sp.description.split('\r\n* ') if i.strip().startswith(f'[{d.base}]')]
            my_description = [i.split(":")[-1] for i in my_description]         
            #print("---",my_description)
            if my_description:
                sp.description = ''.join(my_description)
            #else:
                #print("---",sp.description)
            #except IndexError:   
                #print(sp.expression)
            sp.description = f'**{sp.description.strip()}**  {expect_note}'
            #print(sp.description)
            #display(Markdown(sp.description))
            

  

            #convert to dict since model can't handle primitive extensions
            sp_dict = sp.as_json()

            sp_dict['multipleOr'] = False if d.multipleOr in none_list else True
            sp_dict['_multipleOr'] = sp_expectation(d.multipleOr_conf)
            
            sp_dict['multipleAnd'] = False if d.multipleAnd in none_list else True
            sp_dict['_multipleAnd'] = sp_expectation(d.multipleAnd_conf)

            try:
                sp_dict['_modifier'] = []
                for m in sp_dict['modifier']: # list all modifiers in sp and assign an expectation.
                    if d.shall_modifier not in none_list and m in d.shall_modifier.split(','):
                       sp_dict['_modifier'].append(sp_expectation('SHALL'))
                    elif  d.should_modifier not in none_list and m in d.should_modifier.split(','):
                        sp_dict['_modifier'].append(sp_expectation('SHOULD'))               
                    else:
                        sp_dict['_modifier'].append(sp_expectation('MAY'))
            except KeyError:
                del(sp_dict['_modifier'])

            try:
                sp_dict['_comparator'] = []
                for m in sp_dict['comparator']: # list all comparators in sp and assign an expectation.
                   if d.shall_comparator not in none_list and m in d.shall_comparator.split(','):
                       sp_dict['_comparator'].append(sp_expectation('SHALL'))
                   elif  d.should_comparator not in none_list and m in d.should_comparator.split(','):
                        sp_dict['_comparator'].append(sp_expectation('SHOULD'))               
                   else:
                        sp_dict['_comparator'].append(sp_expectation('MAY'))
            except KeyError:
                del(sp_dict['_comparator'])

            if d.shall_chain not in none_list:
               sp_dict['chain'] = d.shall_chain.split(',')
               sp_dict['_chain'] = [sp_expectation('SHALL') for c in d.shall_chain.split(',')]

            if d.should_chain not in none_list:
               sp_dict['chain'] = d.should_chain.split(',')
               sp_dict['_chain'] = [sp_expectation('SHALL') for c in d.should_chain.split(',')]

            print(f'======================= SP = {sp_dict["id"]} =====================')
            #print(dumps(sp_dict,indent=4))
                  
            #================ add narrative =======================
            template = env.get_template(md_template[3])   
            rendered = template.render(sp=sp_dict)
    
            display(HTML(rendered))
            
            parser = etree.XMLParser(remove_blank_text=True)
            root = etree.fromstring(rendered, parser=parser)

            div = (etree.tostring(root[1][0], encoding='unicode', method='html'))

            narr = N.Narrative()
            narr.status = 'generated'
            narr.div = div
                  
            sp_dict['text'] = narr.as_json()
      
                  
            # ================ save files as resource ======================
           #save in ig_source folder
            write_path = '' #comment out to save in ig
       
            path = Path.cwd() / write_path / 'resources' / f'searchparameter-{sp_dict["id"]}.json'
            if d.base not in skip_types:    #don't write test types  
                path.write_text(dumps(sp_dict,indent=4))
            sp_list.append(sp_dict)

### Validate

- validate sps if validate_flag variable is set to True

In [ ]:

if validate_flag:
    for i in sp_list:
        print(f'Validating {i["id"]}...........')
        r = validate(i)
        display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3>\
                     {r.json()["text"]["div"]}'))
else:
    print(f'Validation Flag is set to {validate_flag}')

## Create Quick Start pages using Jinja
 
- spreadsheet for sp and combos

### Sort out what Searchs are published in Quick Start

-  all single SP with display = true ( SHALLs or SHOULDs)
-  all SHALLs or SHOULDs combos
- ignore all that begin with !

In [ ]:
search_type = dict(
    token = '{system|}[code]',
    id = '[id]',
    reference = '{Type/}[id]',
    string = '[string]',
    uri = '[uri]',
    date = '[date]',
    )
#  add sps not in search_profiles to search_profiles (combo_list)
singles = {i.base for i in data}-{i for i in search_profiles.values()} 
singles_dict = {i.profile:i.base for i in data if i.base in singles}
#pprint(singles_dict)
#pprint(search_profiles)
search_profiles.update(singles_dict)

In [ ]:
pprint(search_profiles)

###  The actual rendering of a markdown Quick Start page

In [ ]:
template = env.get_template(md_template[0])



#print(r_type)   
for profile,type in search_profiles.items():  # preprocess the for jinja templates

    sp = [d for d in data if d.base == type]
    sp_combos = [d for d in combo_data if d.profile == profile]
    #print(sp_combos[0].description)
    mods= {}
    rels = {}
    for s in sp:
            l1=s.shall_modifier.split(',') if s.shall_modifier else []
            l2=s.should_modifier.split(',') if s.should_modifier else []
            l3=s.shall_comparator.split(',') if s.shall_comparator else []
            l4=s.should_comparator.split(',') if s.should_comparator else []
            l5=s.shall_chain.split(',') if s.shall_chain else []
            l6=s.should_chain.split(',') if s.should_chain else []
            l7=s.shall_include.split(',') if s.shall_include else []
            l8=s.should_include.split(',') if s.should_include else []
            shall_multipleAnd= s.multipleAnd =='Y' and s.multipleAnd_conf =='SHALL'
            should_multipleAnd=s.multipleAnd =='Y' and s.multipleAnd_conf =='SHOULD'
            shall_multipleOr=s.multipleOr =='Y' and s.multipleOr_conf =='SHALL'
            should_multipleOr=s.multipleOr =='Y' and s.multipleOr_conf =='SHOULD'
            mods[s.code] = (
                l1+l2,
                l3+l4,
                l1,
                l2,
                l3,
                l4,
                l5,
                l6,
                l7,
                l8,
                shall_multipleAnd,
                should_multipleAnd,
                shall_multipleOr,
                should_multipleOr,
                shall_multipleAnd or should_multipleAnd,
                shall_multipleOr or should_multipleOr,
                   )
            #pprint(mods)
            ''' 0,1 MODS AND COMPS 2,3 mods,
            4.5 comps, 6,7 chains, 8,9 includes 10,11 multAnd, 12,13 multOr, 14 multAnds, 15 multOrs'''
            rels[s.code] = s.rel_url.replace('_','')
            #pprint(rels)

    shalls = "SHALL" in [i.base_conf for i in sp if i.display] + [i.combo_conf for i in sp_combos if i.profile == profile]  # TODO need to search both singles and combos
    shoulds = "SHOULD" in [i.base_conf for i in sp if i.display] + [i.combo_conf for i in sp_combos if i.profile == profile]

    #print(f'''====== type ========
    #{type}
    #====== sp ========
    #{sp}
    #======= search_type =======
    #{search_type}
    #====== sp_combos ========
    #{sp_combos}
    #===== rels =========
    #{rels}
    #''')
    

    search_md = template.render(
                    r_type=type,
                    sp=sp,
                    search_type=search_type,
                    combos=sp_combos,
                    shalls=shalls,
                    shoulds=shoulds,
                    mods = mods,
                    rels = rels,
                    )

    print(f'========={profile} {type} ==============')
    search_md = search_md.replace('\n\n\n', '\n\n') #clean up line feeds
    display(Markdown(search_md))
    # save
    if d.base not in skip_types:
        #write_path = '' #comment out to save in ig
        write_path = "//ERICS-AIR-2/ehaas/Documents/FHIR/US-Core-R4/input/"
        path = Path.cwd() / write_path / 'intro-notes' / f'StructureDefinition-{profile.split("/")[-1]}-notes.md'
        path.write_text(search_md)

### Create Markdown Text for SearchParameters Page

- Using Jinja2 Template create markdown file for searchparameters page
- Use spreadsheet as source

In [ ]:
for d in data:
    if '!' in d.base:
        print(d.base,True)

# get id for each row in spreadsheet:
r_list={d.base for d in data if '!' not in d.base and d.base not in skip_types}
d_tup = []
for d in data:
    if d.base in r_list:
        sp_id= [sp['id'] for sp in sp_list if d.base in sp['base'] and sp['code'] in [d.code,'id'] ]
        d_tup.append((sp_id[0] if sp_id else None,d))

template = env.get_template(md_template[1])   

searchparameters_md = template.render(sp_list=d_tup,r_list=sorted(r_list))

display(Markdown(searchparameters_md))
# save in pages folder
write_path = '' #comment out to save in ig
path = Path.cwd() / write_path / 'pagecontent' / 'searchparameters.md'
path.write_text(searchparameters_md)